In [944]:
import numpy as np
import cv2
import collections
import operator
import copy
from matplotlib import pyplot as plt

In [977]:
#fail: 12,15,20,27,36,48
#fail by start point: 25,28
img = cv2.imread('train/shapes_train2018/1011.jpeg')
h = img.shape[0]
w = img.shape[1]
egdeList = []

In [978]:
def denoising(img):
    return cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)

In [979]:
def kmeans(img, K):
    
    Z = dst.reshape((-1,3))
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((dst.shape))
    return res2


In [980]:
def countPixel(img):
    h = img.shape[0]
    w = img.shape[1]
    my_dict = {}
    count = 0
    a = []
    for i in range(h):
        for j in range(w):
            count += 1
            b = img[i,j]
            temp = []
            for l in range(3):
                temp += [b[l]]
            a.append(temp)
    for i in a:
            i = str(i)
            if i in my_dict:
                my_dict[i] += 1
            else:
                my_dict[i] = 1
    sorted_dict = sorted(my_dict.items(), key=operator.itemgetter(1))
    return sorted_dict

In [981]:
def findBackgroundContour(image, kgraph, backgroundcolor):
    h = image.shape[0]
    w = image.shape[1]
    
    for i in range(w):
        if (not(kgraph[0,i][0] == int(backgroundcolor[0]) and kgraph[0,i][1] == int(backgroundcolor[1]) and kgraph[0,i][2] == int(backgroundcolor[2]))):
            print("we have top line pixels")
            image[0,i][0] = 0
            image[0,i][1] = 0
            image[0,i][2] = 0
    for i in range(w):
        if (not(kgraph[h-1,i][0] == int(backgroundcolor[0]) and kgraph[h-1,i][1] == int(backgroundcolor[1]) and kgraph[h-1,i][2] == int(backgroundcolor[2]))):
            print("we have bot line pixels")
            image[h-1,i][0] = 0
            image[h-1,i][1] = 0
            image[h-1,i][2] = 0
    for i in range(h):
        if (not(kgraph[i,0][0] == int(backgroundcolor[0]) and kgraph[i,0][1] == int(backgroundcolor[1]) and kgraph[i,0][2] == int(backgroundcolor[2]))):
            print("we have left line pixels")
            image[0,i][0] = 0
            image[0,i][1] = 0
            image[0,i][2] = 0 
    for i in range(h):
        if (not(kgraph[i,w-1][0] == int(backgroundcolor[0]) and kgraph[i,h-1][1] == int(backgroundcolor[1]) and kgraph[i,h-1][2] == int(backgroundcolor[2]))):
            print("we have right line pixels")
            image[i,w-1][0] = 0
            image[i,w-1][1] = 0
            image[i,w-1][2] = 0
    return

In [982]:
def Binarize(Color, res):

    copyImage = copy.copy(res)
    tempX = 0
    tempY = 0
    for i in range(h):
        for j in range(w):
            if (copyImage[i,j][0] != int(Color[0]) and copyImage[i,j][1] != int(Color[1]) and copyImage[i,j][2] != int(Color[2])):
                copyImage[i,j][0] = 0
                copyImage[i,j][1] = 0
                copyImage[i,j][2] = 0
    #print(copyImage[5,64])
    for i in range(h):
        for j in range(w):
            #print(i,j)
            if (copyImage[i,j][0] == int(Color[0]) and copyImage[i,j][1] == int(Color[1]) and copyImage[i,j][2] == int(Color[2])):
                tempX = i
                tempY = j
                
                if (i == 0 and j == 0):
                    if (
                        (copyImage[i+1,j+1][0] == int(Color[0]) and
                        copyImage[i+1,j+1][1] == int(Color[1]) and
                        copyImage[i+1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (i == 0 and j == w-1):
                    if (
                        (copyImage[i+1,j-1][0] == int(Color[0]) and
                        copyImage[i+1,j-1][1] == int(Color[1]) and
                        copyImage[i+1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (i == h-1 and j == 0):
                    if (
                        (copyImage[i-1,j+1][0] == int(Color[0]) and
                        copyImage[i-1,j+1][1] == int(Color[1]) and
                        copyImage[i-1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (i == h-1 and j == w-1):
                    if (
                        (copyImage[i-1,j-1][0] == int(Color[0]) and
                        copyImage[i-1,j-1][1] == int(Color[1]) and
                        copyImage[i-1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (i > 0 and i < h-1 and j == 0):
                    if (
                        (copyImage[i-1,j+1][0] == int(Color[0]) and
                        copyImage[i-1,j+1][1] == int(Color[1]) and
                        copyImage[i-1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j+1][0] == int(Color[0]) and
                        copyImage[i+1,j+1][1] == int(Color[1]) and
                        copyImage[i+1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (i > 0 and i < h-1 and j == w-1):
                    if (
                        (copyImage[i-1,j-1][0] == int(Color[0]) and
                        copyImage[i-1,j-1][1] == int(Color[1]) and
                        copyImage[i-1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j-1][0] == int(Color[0]) and
                        copyImage[i+1,j-1][1] == int(Color[1]) and
                        copyImage[i+1,j-1][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                if (j > 0 and j < h-1 and i == w-1):
                    if (
                        (copyImage[i-1,j-1][0] == int(Color[0]) and
                        copyImage[i-1,j-1][1] == int(Color[1]) and
                        copyImage[i-1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j+1][0] == int(Color[0]) and
                        copyImage[i-1,j+1][1] == int(Color[1]) and
                        copyImage[i-1,j+1][2] == int(Color[2]))
                    ):
                        return [i,j]
        
                if (j > 0 and j < h-1 and i == 0):
                    if (
                        (copyImage[i+1,j-1][0] == int(Color[0]) and
                        copyImage[i+1,j-1][1] == int(Color[1]) and
                        copyImage[i+1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2])) 
                        or
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j+1][0] == int(Color[0]) and
                        copyImage[i+1,j+1][1] == int(Color[1]) and
                        copyImage[i+1,j+1][2] == int(Color[2]))
                    ):
                        return [i,j]
                    
                else:
                    if (
                        (copyImage[i+1,j][0] == int(Color[0]) and
                        copyImage[i+1,j][1] == int(Color[1]) and
                        copyImage[i+1,j][2] == int(Color[2]))
                        or
                        (copyImage[i,j-1][0] == int(Color[0]) and
                        copyImage[i,j-1][1] == int(Color[1]) and
                        copyImage[i,j-1][2] == int(Color[2])) 
                        or
                        (copyImage[i+1,j-1][0] == int(Color[0]) and
                        copyImage[i+1,j-1][1] == int(Color[1]) and
                        copyImage[i+1,j-1][2] == int(Color[2]))
                        or
                        (copyImage[i,j+1][0] == int(Color[0]) and
                        copyImage[i,j+1][1] == int(Color[1]) and
                        copyImage[i,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i+1,j+1][0] == int(Color[0]) and
                        copyImage[i+1,j+1][1] == int(Color[1]) and
                        copyImage[i+1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j+1][0] == int(Color[0]) and
                        copyImage[i-1,j+1][1] == int(Color[1]) and
                        copyImage[i-1,j+1][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j][0] == int(Color[0]) and
                        copyImage[i-1,j][1] == int(Color[1]) and
                        copyImage[i-1,j][2] == int(Color[2]))
                        or
                        (copyImage[i-1,j-1][0] == int(Color[0]) and
                        copyImage[i-1,j-1][1] == int(Color[1]) and
                        copyImage[i-1,j-1][2] == int(Color[2]))
                    ):
                        return [i,j]
                #print(copyImage[i,j])
                #cv2.imshow("copy", copyImage)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                
                #return [i,j]
                
                
    #print(tempX, tempY)
    #print(copyImage[114,78]
    #print(Color)
    #cv2.imshow("copy", copyImage)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #print(counter)

    

In [983]:
def Following_Algorithm(points, res):
    tempX = 0
    tempY = 0
    copyImage = copy.copy(res)
    tempList = []
    for i in points:
        tempX = i[0]
        tempY = i[1]
        rememberPoint = [tempX, tempY]
        flag = True
        c = [tempX, tempY]
        #if (tempX == 0 || tempX == h-1):
        #    for j in range(tempX, w):
        #        if (copyImage[j,tempY].all() == copyImage[tempX, tempY].all()):
        #            tempX = j
        #            c = [tempX, tempY]

        #if (tempX - 1 < 0):
        tempList.append(c)
        p = 0
        if (c[0] == 0  or c[0] == h-2 or c[1] == 0 or c[1] == w-2):
            new = touchborder(c, copyImage, rememberPoint)
            c = new[0]
            p = new[1]
        print("remember",rememberPoint)
        while(flag):
            for n in range(p, 14):
                #print("p value", p)
                #print(tempList)
                if n > 7:
                    n = n - 8
                tempPoint = N_eight(n,c)
                #print("tempPoint", tempPoint, n)
                
                if ((copyImage[tempPoint[0], tempPoint[1]])[0] == (copyImage[rememberPoint[0], rememberPoint[1]])[0]
                   and (copyImage[tempPoint[0], tempPoint[1]])[1] == (copyImage[rememberPoint[0], rememberPoint[1]])[1]
                   and (copyImage[tempPoint[0], tempPoint[1]])[2] == (copyImage[rememberPoint[0], rememberPoint[1]])[2]
                   ):
                #if (np.all(copyImage[tempPoint[0], tempPoint[1]], copyImage[rememberPoint[0], rememberPoint[1]])):
                    #print("nvalue",n)
                    #ccopy = copy.deepcopy(c)
                    #print("copyhere", ccopy)
                    #print("tempPointEnter", tempPoint, n)
                    #c = N_eight(n, c)
                    c = tempPoint
                    print("c_enter",c)
                    if (n == 0):
                        p = n
                    else:
                        p = n-1   
                    if(tempPoint != rememberPoint):
                        #if touch the border
                        if (c[0] == 0 or c[0] == h-2 or c[1] == 0 or c[1] == w-2):
                            new = touchborder(c, copyImage, rememberPoint)
                            if (new[2] == 1):
                                flag = False
                            c = new[0]
                            p = new[1]
                            print("c,p",c,p)
                    
                    tempList.append(c)
                    #print(tempList)
                    break
            if (c[0] == rememberPoint[0] and c[1] == rememberPoint[1]):
                flag = False
        egdeList.append(tempList)
#    print(points)
#    print(tempList)

    return

In [984]:
def N_eight(n, c):
    if n > 7:
        n = n - 8
    i = c[0]
    j = c[1]
    if n == 0 or n == 8:
        return[i - 1, j]
    elif n == 1 or n == 9:
        return [i - 1, j + 1]
    elif n == 2 or n == 10:
        return [i, j + 1]
    elif n == 3 or n == 11:
        return [i + 1, j + 1]
    elif n ==4 or n == 12:
        return [i + 1, j]
    elif n ==5 or n == 13:
        return [i + 1, j - 1]
    elif n ==6:
        return [i, j - 1]
    elif n ==7:
        return [i - 1, j - 1]

In [985]:
def touchborder(c, copyImage, rememberPoint):
    if (c[0] == 0):
        #top?
        p = 2
        while ((copyImage[c[0], c[1]])[0] == (copyImage[rememberPoint[0], rememberPoint[1]])[0] and (copyImage[c[0], c[1]])[1] == (copyImage[rememberPoint[0], rememberPoint[1]])[1] and (copyImage[c[0], c[1]])[2] == (copyImage[rememberPoint[0], rememberPoint[1]])[2]):
            if (c[0] == rememberPoint[0] and c[1] == rememberPoint[1]):
                return [c,p,1]
                break
            if(c[1] == 0):
                return [c,4,0]
            c[1] = c[1] + 1
        c[1] = c[1] - 1
        return [c,p,0]
    elif (c[1] == 0):
        #left
        p = 0
        while ((copyImage[c[0], c[1]])[0] == (copyImage[rememberPoint[0], rememberPoint[1]])[0] and (copyImage[c[0], c[1]])[1] == (copyImage[rememberPoint[0], rememberPoint[1]])[1] and (copyImage[c[0], c[1]])[2] == (copyImage[rememberPoint[0], rememberPoint[1]])[2]):
            if (c[0] == rememberPoint[0] and c[1] == rememberPoint[1]):
                return [c,p,1]
                break
            if(c[0] == w-2):
                return [c,7,0]
            c[0] = c[0] - 1
        c[0] = c[0] + 1
        return [c,p,0]
    elif (c[1] == w-2):
        #right
        p = 4
        while ((copyImage[c[0], c[1]])[0] == (copyImage[rememberPoint[0], rememberPoint[1]])[0] and (copyImage[c[0], c[1]])[1] == (copyImage[rememberPoint[0], rememberPoint[1]])[1] and (copyImage[c[0], c[1]])[2] == (copyImage[rememberPoint[0], rememberPoint[1]])[2]):
            if (c[0] == rememberPoint[0] and c[1] == rememberPoint[1]):
                return [c,p,1]
                break
            if(c[0] == w-2):
                return [c,7,0]
            c[0] = c[0] + 1
            print("c",c)
        c[0] = c[0] - 1
        #print("c-1",c)
        return [c,p,0]
    elif (c[0] == h-2):
        #bottom
        p = 6
        while ((copyImage[c[0], c[1]])[0] == (copyImage[rememberPoint[0], rememberPoint[1]])[0] and (copyImage[c[0], c[1]])[1] == (copyImage[rememberPoint[0], rememberPoint[1]])[1] and (copyImage[c[0], c[1]])[2] == (copyImage[rememberPoint[0], rememberPoint[1]])[2]):
            if (c[0] == rememberPoint[0] and c[1] == rememberPoint[1]):
                return [c,p,1]
                break
            if(c[1] == 0):
                return [c,0,0]
            c[1] = c[1] - 1
        c[1] = c[1] + 1
        return [c,p,0]

In [986]:
dst = denoising(img)

a = countPixel(img)

counter = 0

#can be improved
for i in range (1,6):
    if (a[-i][1] >= 220):
        counter += 1

res = kmeans(dst, counter)

c = countPixel(res)

#can be improved
for i in range(len(c)):
    if c[i][1] <= 560:
        counter -= 1

res = kmeans(dst, counter)

c = countPixel(res)

ColorList = []
for i in c:
    temp = i[0]
    temp = temp.replace("[", "")
    temp = temp.replace("]", "")
    temp = temp.replace(" ", "")
    temp = temp.split(",")
    ColorList.append(temp)
        
        
points = []        

ColorList = ColorList[:-1]

for i in ColorList:
    points.append(Binarize(i,res))
print(points)
Following_Algorithm(points, res)

BackgroudColor = countPixel(res)[-1][0]

BackgroudColor = BackgroudColor.replace("[", "")
BackgroudColor = BackgroudColor.replace("]", "")
BackgroudColor = BackgroudColor.replace(" ", "")
BackgroudColor = BackgroudColor.split(",")


#findBackgroundContour(img, res, BackgroudColor)

print(egdeList)

#cv2.imshow("ori", img)
#cv2.imshow("aft", dst)
#cv2.imshow("res", res)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

[[1, 48], [77, 73]]
remember [1, 48]
c_enter [2, 49]
c_enter [2, 50]
c_enter [2, 51]
c_enter [2, 52]
c_enter [2, 53]
c_enter [2, 54]
c_enter [3, 55]
c_enter [3, 56]
c_enter [3, 57]
c_enter [4, 58]
c_enter [4, 59]
c_enter [5, 60]
c_enter [5, 61]
c_enter [6, 62]
c_enter [7, 63]
c_enter [8, 64]
c_enter [9, 65]
c_enter [10, 66]
c_enter [11, 67]
c_enter [12, 68]
c_enter [13, 68]
c_enter [14, 69]
c_enter [15, 69]
c_enter [16, 70]
c_enter [17, 70]
c_enter [18, 70]
c_enter [19, 71]
c_enter [20, 71]
c_enter [21, 71]
c_enter [22, 71]
c_enter [23, 71]
c_enter [24, 71]
c_enter [25, 72]
c_enter [26, 71]
c_enter [27, 71]
c_enter [28, 71]
c_enter [29, 71]
c_enter [30, 71]
c_enter [31, 71]
c_enter [32, 70]
c_enter [33, 70]
c_enter [34, 70]
c_enter [35, 69]
c_enter [36, 69]
c_enter [37, 68]
c_enter [38, 68]
c_enter [39, 67]
c_enter [40, 66]
c_enter [41, 65]
c_enter [42, 64]
c_enter [43, 63]
c_enter [44, 62]
c_enter [45, 61]
c_enter [45, 60]
c_enter [46, 59]
c_enter [46, 58]
c_enter [47, 57]
c_enter [47